# EDA

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import linear_model
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
X_train = pd.read_csv("data/train_features.csv")
y_train = pd.read_csv("data/train_targets.csv")
X_test = pd.read_csv("data/test_features.csv")